In [1]:
!pip install transformers
!pip install peft


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# Load the base model and tokenizer
base_model_id = "unsloth/Llama-3.2-1B-Instruct"
fine_tuned_model_id = "Ali7865253/llama-3.2-1B-Lora-Fine_Tune-FineTome"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_id)

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(base_model_id)

# Load the fine-tuned LoRA model
model = PeftModel.from_pretrained(base_model, fine_tuned_model_id)

# Check if a GPU is available and move the model to the GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Ensure the tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Function to generate responses
def generate_response(instruction, query):
    prompt = f"Instruction: {instruction}\nUser Query: {query}\nAI Response:"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        temperature=1.5,
        top_p=0.9,
        do_sample=True,
        no_repeat_ngram_size=3  # Prevent n-gram repetition
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Remove the prompt from the response to avoid repetition
    response = response.split('AI Response:')[-1].strip()

    # Break the response into multiple lines for better readability
    organized_response = "\n".join([f"- {line.strip()}" for line in response.split('. ') if line.strip()])

    return organized_response

# Example usage
instruction = "Provide helpful, empathetic mental health advice."
queries = [
    "having severe headache what should i do?",
    "what are symptoms i am having flu?"
]

for query in queries:
    print(f"User: {query}\n")
    response = generate_response(instruction, query)
    print(f"AI:\n{response}\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/927 [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


User: having severe headache what should i do?



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


AI:
- I'll do my best to give a helpful and honest answer
- If your are struggling with an overwhelming emotional or psychological pain and in case of crisis call or contact one your health center's main number - there can help to guide you through this or an individual crisis line.

User Response to Mental health:

For all mental health issues, try these tips to deal with it - Take deep breaths, think of three things to improve your mental well-being in this time of stress
- Be open with your feelings, practice mindfulness or just focus on being in this moment, give yourself a little compassion for

User: what are symptoms i am having flu?

AI:
- Congrats on completing the task! Pneumonia can also cause similar symptoms
- According to various resources, some common symptoms, including the flu, you could face with pneumonia include fever for prolonged periods of more than 2 days, increased pressure in the chest or the lungs (shortness of breath or difficult breathing)
- This may result